# Process
- Import the Link Mobility report file that contains the codes, emails, error message
- Take benefs that weren't delivered
- Match those benefs with data from production
- Output a CSV file with the folder id, email, code and link to Demarches Simplifiees

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
wave_report_filepath = os.getenv('WAVE_REPORT_FILEPATH')
production_benefs_with_folders_id_filepath = os.getenv('PRODUCTION_BENEF_WITH_FOLDERS_ID')

df = pd.read_csv(wave_report_filepath,on_bad_lines='skip', sep=';', dtype=str, engine="c", keep_default_na=False, encoding='utf-8')
df.rename(columns={'CODE': 'id_psp', 'EMAIL': 'email'}, inplace=True)

In [ ]:
df['NM_ERROR_NAME'].value_counts()

In [ ]:
mask_not_delivered = (df['NM_ERROR_NAME'] == 'Non D�livr�') | (df['NM_ERROR_NAME'] == 'Dest. Inconnu') | (df['NM_ERROR_NAME'] == 'Bo�te non dispo.') | (df['NM_ERROR_NAME'] == 'Message Refus�')
df_codes_not_delivered = df[mask_not_delivered]

In [ ]:
df_codes_not_delivered

In [ ]:
df_wave_2 = pd.read_csv(production_benefs_with_folders_id_filepath, on_bad_lines='skip', sep=';', dtype=str, engine="c", keep_default_na=False, encoding="utf-8")

df_dossiers_not_delivered = pd.merge(df_wave_2, df_codes_not_delivered, how='inner', on='id_psp', suffixes=('', '_not_delivered'))
df_dossiers_not_delivered['folder_link'] = f'https://www.demarches-simplifiees.fr/procedures/126472/a-suivre/dossiers/' + df_dossiers_not_delivered['dossier_id']
df_dossiers_not_delivered[['id_psp', 'dossier_id', 'email', 'folder_link']].to_csv('./folders-not-delivered.csv', sep=';', encoding='utf-8', index=False)